# The Encoder-Decoder Architecture

In general seq2seq problems like machine translation, _inputs and outputs are of varying lengths that are unaligned_.

The standard approach to handling this sort of data is to design an encoder-decoder architecture consisting of two major components: 

- __An encoder:__ that takes a variable-length sequence as input, 
- __And a decoder:__ that acts as a _conditional language model_, taking in the encoded input and the leftwards context of the target sequence and predicting the subsequent token in the target sequence.

Let’s take machine translation from English to French as an example. Given an input sequence in English: “They”, “are”, “watching”, “.”, this encoder-decoder architecture:

* first encodes the variable-length input into a state, 
* then decodes the state to generate the translated sequence, token by token, as output: “Ils”, “regardent”, “.”. 


In [2]:
from torch import nn
from d2l import torch as d2l

## Encoder

In the encoder interface, we just specify that the encoder takes variable-length sequences as input X. 

In [3]:
class Encoder(nn.Module):  #@save
    """The base encoder interface for the encoder-decoder architecture."""
    def __init__(self):
        super().__init__()

    # Later there can be additional arguments (e.g., length excluding padding)
    def forward(self, X, *args):
        raise NotImplementedError

## Decoder

In the following decoder interface, we add an additional init_state method to convert the encoder output (enc_all_outputs) into the encoded state. Note that this step may require extra inputs, such as the valid length of the input. To generate a variable-length sequence token by token, every time the decoder may map an input (e.g., the generated token at the previous time step) and the encoded state into an output token at the current time step.

In [5]:
class Decoder(nn.Module):  #@save
    """The base decoder interface for the encoder-decoder architecture."""
    def __init__(self):
        super().__init__()

    # Later there can be additional arguments (e.g., length excluding padding)
    def init_state(self, enc_all_outputs, *args):
        raise NotImplementedError

    def forward(self, X, state):
        raise NotImplementedError

## Encoder-Decoder

In the forward propagation, the output of the encoder is used to produce the encoded state, and this state will be further used by the decoder as one of its input.

In [6]:
class EncoderDecoder(d2l.Classifier):  #@save
    """The base class for the encoder-decoder architecture."""
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, enc_X, dec_X, *args):
        enc_all_outputs = self.encoder(enc_X, *args)
        dec_state = self.decoder.init_state(enc_all_outputs, *args)
        # Return decoder output only
        return self.decoder(dec_X, dec_state)[0]

Encoder-decoder architectures can handle inputs and outputs that both consist of variable-length sequences and thus are suitable for seq2seq problems such as machine translation. The encoder takes a variable-length sequence as input and transforms it into a state with a fixed shape. The decoder maps the encoded state of a fixed shape to a variable-length sequence.